In [ ]:
# pip install Flask requests beautifulsoup4

# app.py
from flask import Flask, render_template
import requests
from bs4 import BeautifulSoup

app = Flask(__name__)


def scrape_youtube():
    url = 'https://www.youtube.com/feed/trending'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    videos = []
    for video in soup.find_all('a', class_='yt-uix-tile-link'):
        videos.append({
            'title': video.text,
            'url': 'https://www.youtube.com' + video['href']
        })
    return videos

def scrape_amazon():
    url = 'https://www.amazon.com/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    products = []
    for product in soup.find_all('span', class_='a-size-medium'):
        products.append(product.text)
    return products

@app.route('/')
def index():
    youtube_videos = scrape_youtube()
    amazon_products = scrape_amazon()
    return render_template('index.html', youtube_videos=youtube_videos, amazon_products=amazon_products)

if __name__ == '__main__':
    app.run(debug=True)

    
# templates/index.html 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Scraped Data</title>
</head>
<body>
    <h1>YouTube Trending Videos</h1>
    <ul>
        {% for video in youtube_videos %}
        <li><a href="{{ video.url }}" target="_blank">{{ video.title }}</a></li>
        {% endfor %}
    </ul>
    <h1>Amazon Top Products</h1>
    <ul>
        {% for product in amazon_products %}
        <li>{{ product }}</li>
        {% endfor %}
    </ul>
</body>
</html>


In [ ]:
# pip install Flask requests

# app.py
from flask import Flask, render_template, request
import requests

app = Flask(__name__)


API_KEY = 'OPENWEATHERMAP_API_KEY'
API_ENDPOINT = 'http://api.openweathermap.org/data/2.5/weather'


@app.route('/')
def index():
    return render_template('index.html')

@app.route('/weather', methods=['POST'])
def weather():
    city = request.form['city']
    params = {
        'q': city,
        'appid': API_KEY,
        'units': 'metric'  
    }
    response = requests.get(API_ENDPOINT, params=params)
    weather_data = response.json()

    if weather_data['cod'] == 200:
        weather_info = {
            'city': weather_data['name'],
            'temperature': weather_data['main']['temp'],
            'description': weather_data['weather'][0]['description']
        }
    else:
        weather_info = None

    return render_template('weather.html', weather_info=weather_info)

if __name__ == '__main__':
    app.run(debug=True)

    
    
# templates/index.html 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weather App</title>
</head>
<body>
    <h1>Weather App</h1>
    <form action="/weather" method="post">
        <label for="city">Enter city name:</label>
        <input type="text" id="city" name="city" required>
        <button type="submit">Get Weather</button>
    </form>
</body>
</html>
  
    
    
# templates/weather.html 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Weather</title>
</head>
<body>
    {% if weather_info %}
    <h1>Weather in {{ weather_info.city }}</h1>
    <p>Temperature: {{ weather_info.temperature }}°C</p>
    <p>Description: {{ weather_info.description }}</p>
    {% else %}
    <p>Weather data not available for the provided city.</p>
    {% endif %}
    <a href="/">Back to home</a>
</body>
</html>


In [ ]:
#pip install Flask Flask-OAuthlib requests requests-oauthlib
# app.py
from flask import Flask, redirect, url_for, session
from flask_oauthlib.client import OAuth

app = Flask(__name__)
app.secret_key = 'your_secret_key'

# OAuth configuration for Google
oauth = OAuth(app)
google = oauth.remote_app(
    'google',
    consumer_key='your_google_client_id',
    consumer_secret='your_google_client_secret',
    request_token_params={
        'scope': 'email'
    },
    base_url='https://www.googleapis.com/oauth2/v1/',
    request_token_url=None,
    access_token_method='POST',
    access_token_url='https://accounts.google.com/o/oauth2/token',
    authorize_url='https://accounts.google.com/o/oauth2/auth'
)


facebook = oauth.remote_app(
    'facebook',
    consumer_key='your_facebook_app_id',
    consumer_secret='your_facebook_app_secret',
    request_token_params={'scope': 'email'},
    base_url='https://graph.facebook.com',
    request_token_url=None,
    access_token_url='/oauth/access_token',
    authorize_url='https://www.facebook.com/dialog/oauth'
)

@app.route('/login/google')
def login_google():
    return google.authorize(callback=url_for('authorized_google', _external=True))

@app.route('/login/google/authorized')
def authorized_google():
    resp = google.authorized_response()
    if resp is None or resp.get('access_token') is None:
        return 'Access denied: reason={} error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )
    session['google_token'] = (resp['access_token'], '')
    user_info = google.get('userinfo')
    return 'Logged in as: ' + user_info.data['email']

@google.tokengetter
def get_google_oauth_token():
    return session.get('google_token')

@app.route('/login/facebook')
def login_facebook():
    return facebook.authorize(callback=url_for('authorized_facebook', _external=True))

@app.route('/login/facebook/authorized')
def authorized_facebook():
    resp = facebook.authorized_response()
    if resp is None or resp.get('access_token') is None:
        return 'Access denied: reason={} error={}'.format(
            request.args['error_reason'],
            request.args['error_description']
        )
    session['facebook_token'] = (resp['access_token'], '')
    user_info = facebook.get('/me')
    return 'Logged in as: ' + user_info.data['name']

@facebook.tokengetter
def get_facebook_oauth_token():
    return session.get('facebook_token')

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# app.py
from flask import Flask, request, jsonify

app = Flask(__name__)


user_preferences = {
    'user1': {'action': 5, 'comedy': 3, 'drama': 4, 'horror': 1},
    'user2': {'action': 4, 'comedy': 2, 'drama': 5, 'horror': 3},
  
}

content_data = {
    'movie1': {'action': 4, 'comedy': 2, 'drama': 5, 'horror': 3},
    'movie2': {'action': 3, 'comedy': 4, 'drama': 5, 'horror': 2},

}

def recommend(user_id):
    user_prefs = user_preferences.get(user_id)
    if not user_prefs:
        return []

    scores = {}
    for content_id, content_prefs in content_data.items():
        score = 0
        for genre, rating in user_prefs.items():
            score += rating * content_prefs.get(genre, 0)
        scores[content_id] = score

    recommended_content = sorted(scores, key=scores.get, reverse=True)
    return recommended_content[:5]  

@app.route('/recommendations', methods=['GET'])
def get_recommendations():
    user_id = request.args.get('user_id')
    if not user_id:
        return jsonify({'error': 'User ID is required'}), 400
    
    recommendations = recommend(user_id)
    return jsonify({'recommendations': recommendations})

if __name__ == '__main__':
    app.run(debug=True)
